In [1]:
cd ../

/home/amir/study/langchain_vectordb


/home/amir/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key: str = os.getenv("api_key", "")
base_url: str = os.getenv("base_url", "")

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-5-nano-2025-08-07",
    api_key=api_key,
    base_url=base_url,
)

In [4]:
!pip install distro

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)


In [6]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store = {}
# Напишем функцию, которая возвращает историю диалога по session ID.
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [7]:
# Создаём конфиг для нашего Runnable, чтобы указать ему session_id при вызове
config = {"configurable": {"session_id": "1"}}

llm_with_history = RunnableWithMessageHistory(
    llm,
    get_session_history
)
# к вызову добавляем параметр config
llm_with_history.invoke("Привет, ChatGPT! Меня зовут Иван. Как дела?", config=config).content

'Привет, Иван! У меня всё хорошо, спасибо. Как твои дела? Чем могу помочь сегодня — могу потренировать русский язык, рассказать что-то на интересную тему, помочь с задачами, написать текст или разобрать код. Что хочешь сделать?'

In [8]:
# В store появилась первая пара запрос-ответ, и ключ "1" - session_id
print(store['1'])

Human: Привет, ChatGPT! Меня зовут Иван. Как дела?
AI: Привет, Иван! У меня всё хорошо, спасибо. Как твои дела? Чем могу помочь сегодня — могу потренировать русский язык, рассказать что-то на интересную тему, помочь с задачами, написать текст или разобрать код. Что хочешь сделать?


In [9]:
llm_with_history.invoke("Сможешь помочь мне в написании кода на Python?", config=config).content

'Здорово, могу помочь. Опиши, что именно нужно сделать в Python: задача, входные данные, ожидаемый вывод, окружение (версия Python, IDE, OS). Если есть код — пришли его, вместе разберём.\n\nЧтобы ускорить процесс, могу предложить несколько форматов взаимодействия:\n- Напиши задачу, и я дам готовое решение с пояснениями.\n- Пришли свой код — найдём баги, улучшим стиль и читаемость.\n- Объясню концепцию или паттерны (функции, обработка исключений, тесты и т.д.).\n- Сделаем пошаговую отработку задачи: план, минимальный пример, тесты, рефакторинг.\n\nЕсли хочешь сразу попрактиковаться, вот пару простых примеров задач и готовых решений (без слишком сложной логики):\n\n- Сумма элементов списка\n  def sum_list(nums):\n      return sum(nums)\n\n- Факториал (итеративно)\n  def factorial(n):\n      if n < 0:\n          raise ValueError("n должен быть неотрицательным")\n      result = 1\n      for i in range(2, n + 1):\n          result *= i\n      return result\n\n- Уникальные элементы в порядке

In [11]:
llm_with_history.invoke("Как вывести на экран фразу 'Hello, world!'?", config=config).content

'Вот простой пример для Python 3:\n\n- print("Hello, world!")\n\nЕсли нужно вывести фразу с кавычками внутри, используйте эскейп-символы или другой вид кавычек:\n- print("\'Hello, world!\'")  выводит: \'Hello, world!\'\n- print(\'"Hello, world!"\')  выводит: "Hello, world!"\n\nКак запустить:\n- Сохраните файл, например, hello.py, с содержимым print("Hello, world!").\n- Запустите в терминале: python3 hello.py\n\nЕсли нужна версия под Python 2, тогда можно писать: print "Hello, world!" (но Python 2 устарел и стоит использовать Python 3).'

In [13]:
llm_with_history.invoke("Как меня зовут?", config=config).content

'Тебя зовут Иван. \n\nЕсли хочешь, могу называть тебя по имени в рамках этой беседы. Чем могу помочь дальше?'

In [14]:
# Очистить историю диалога сессии можно методом clear()
get_session_history('1').clear()
store

{'1': InMemoryChatMessageHistory(messages=[])}

In [15]:
llm_with_history.invoke("Как меня зовут?", config=config).content

'Не знаю вашего имени по умолчанию. Скажите, как вам удобнее, чтобы я к вам обращался: назовите имя или ник, который использовать в этом чате, и я буду обращаться именно так. Если не хотите называть имя, можно и без него. Замечу, что между сеансами я не запоминаю данные, так что повторно нужно указывать имя в новом разговоре.'

In [17]:
from langchain_core.messages import HumanMessage, AIMessage

get_session_history('1').clear()

user_message = HumanMessage(content="Привет меня зовут Абдулла?")
ai_message = AIMessage(content="Привет, Абдулла. Чем могу помочь?")

get_session_history('1').add_messages([user_message, ai_message])

print(store['1'])

Human: Привет меня зовут Абдулла?
AI: Привет, Абдулла. Чем могу помочь?


In [18]:
llm_with_history.invoke("Как меня зовут?", config=config).content

'Вы — Абдулла. Чем могу помочь?'

In [22]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Создадим шаблон промпта для ассистента
# Оставим в шаблоне "заглушку" - MessagesPlaceholder, в которую будет подставляться история диалога
prompt = ChatPromptTemplate.from_messages([
    ("system", "Ты полезный ассистент, отлично разбирающийся в {topic}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

chain = prompt | llm

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='topic',
    history_messages_key='chat_history',
    
)
chain_with_history.invoke(
{"topic": "математика", "question": "Чему равен синус 30 градусов?"},
config={"configurable": {"session_id": "2"}}).content

'1/2 (0.5).\n\nОбоснование: в треугольнике 30–60–90 отношение противоположной стороны к гипотенузе равно 1:2, значит sin 30° = 1/2. Также на единичной окружности sin(π/6) = 1/2.'

In [23]:
chain_with_history.invoke(
    {"topic": "математика", "question": "А чему равен косинус?"},
    config={"configurable": {"session_id": "2"}}).content

'Косинус зависит от угла.\n\n- Для угла 30°: cos 30° = adjacent/hypotenuse = √3/2 ≈ 0.866.\n- Для угла 60°: cos 60° = 1/2 = 0.5.\n\nНа единичной окружности: cos(π/6) = √3/2, cos(π/3) = 1/2.\n\nЕсли имелось в виду другой угол, скажи какой, уточню.'

In [24]:
print(store['2'])

Human: математика
AI: 1/2 (0.5).

Обоснование: в треугольнике 30–60–90 отношение противоположной стороны к гипотенузе равно 1:2, значит sin 30° = 1/2. Также на единичной окружности sin(π/6) = 1/2.
Human: математика
AI: Косинус зависит от угла.

- Для угла 30°: cos 30° = adjacent/hypotenuse = √3/2 ≈ 0.866.
- Для угла 60°: cos 60° = 1/2 = 0.5.

На единичной окружности: cos(π/6) = √3/2, cos(π/3) = 1/2.

Если имелось в виду другой угол, скажи какой, уточню.
